In [11]:
using JuMP, GLPK, Clp, Cbc

### ⚙️ Inciso 2 - Modelo de Producción, Períodos múltiples

In [12]:
using JuMP, GLPK

# Parámetros
d = [180, 250, 190, 140, 220, 250]  # Demanda mensual
c = [50, 45, 55, 52, 48, 50]        # Costo de producción por ventana
h = [8, 10, 10, 10, 8, 8]           # Costo de almacenamiento por ventana
capacidad = 225                     # Capacidad máxima de producción

# Modelo
modelo = Model(GLPK.Optimizer)

# Variables
@variable(modelo, x[1:6] >= 0)  # Producción
@variable(modelo, s[1:6] >= 0)  # Inventario

# Función objetivo: minimizar el costo total (producción + almacenamiento)
@objective(modelo, Min, sum(c[i] * x[i] for i in 1:6) + sum(h[i] * s[i] for i in 1:6))

# Restricciones de balance de inventario
@constraint(modelo, s[1] == x[1] - d[1])
for i in 2:6
    @constraint(modelo, s[i] == s[i-1] + x[i] - d[i])
end

# Restricciones de capacidad de producción
for i in 1:6
    @constraint(modelo, x[i] <= capacidad)
end

# Resolver el modelo
optimize!(modelo)

# Obtener resultados
produccion_optima = value.(x)
inventario_optimo = value.(s)
costo_total_optimo = objective_value(modelo)

println("Producción óptima: ", produccion_optima)
println("Inventario óptimo: ", inventario_optimo)
println("Costo total óptimo: ", costo_total_optimo)

Producción óptima: [205.0, 225.0, 190.0, 160.0, 225.0, 225.0]
Inventario óptimo: [25.0, 0.0, 0.0, 20.0, 25.0, 0.0]
Costo total óptimo: 61795.0


### 🕰️ Inciso 3 - Modelo de asignación de horarios

In [13]:

model = Model(GLPK.Optimizer)

# x1, x2, x3, x4, x5, x6 (cantidad de buses que comienzan en cada turno)
@variable(model, x1 >= 0, Int)
@variable(model, x2 >= 0, Int)
@variable(model, x3 >= 0, Int)
@variable(model, x4 >= 0, Int)
@variable(model, x5 >= 0, Int)
@variable(model, x6 >= 0, Int)

@objective(model, Min, x1 + x2 + x3 + x4 + x5 + x6)

@constraint(model, x1 + x6 >= 4)   # 12:00 AM - 4:00 AM
@constraint(model, x1 + x2 + x6 >= 8)   # 4:00 AM - 8:00 AM
@constraint(model, x1 + x2 + x3 >= 10)  # 8:00 AM - 12:00 PM
@constraint(model, x3 + x4 >= 12)  # 12:00 PM - 4:00 PM
@constraint(model, x4 + x5 >= 7)   # 4:00 PM - 8:00 PM
@constraint(model, x5 + x6 >= 4)   # 8:00 PM - 12:00 AM


x5 + x6 >= 4

In [14]:
model

A JuMP Model
├ solver: GLPK
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 6
├ num_constraints: 18
│ ├ AffExpr in MOI.GreaterThan{Float64}: 6
│ ├ VariableRef in MOI.GreaterThan{Float64}: 6
│ └ VariableRef in MOI.Integer: 6
└ Names registered in the model
  └ :x1, :x2, :x3, :x4, :x5, :x6

In [15]:
optimize!(model)

# Imprimir resultados
println("Estado de la solución: ", termination_status(model))
println("x1 = ", value(x1))
println("x2 = ", value(x2))
println("x3 = ", value(x3))
println("x4 = ", value(x4))
println("x5 = ", value(x5))
println("x6 = ", value(x6))
println("Cantidad total de autobuses = ", objective_value(model))

Estado de la solución: OPTIMAL
x1 = 0.0
x2 = 4.0
x3 = 6.0
x4 = 7.0
x5 = 0.0
x6 = 4.0
Cantidad total de autobuses = 21.0


### 🏘️ Inciso 4 - Modelo de renovación urbana

In [16]:
model = Model(Clp.Optimizer)

@variable(model, x1 >= 0)  # Sencillas
@variable(model, x2 >= 0)  # Dobles
@variable(model, x3 >= 0)  # Triples
@variable(model, x4 >= 0)  # Cuádruples

# Maximizar los impuestos
@objective(model, Max, 1000*x1 + 1900*x2 + 2700*x3 + 3400*x4)

# Restricción del área disponible
@constraint(model, 0.18*x1 + 0.28*x2 + 0.4*x3 + 0.5*x4 <= 63.75)

# Restricción de proporción: Triples y cuádruples >= 25% del total
@constraint(model, x3 + x4 >= 0.25 * (x1 + x2 + x3 + x4))

# Restricción de proporción: Sencillas >= 10% del total
@constraint(model, x1 >= 0.1 * (x1 + x2 + x3 + x4))

# Restricción del financiamiento
@constraint(model, 50000*x1 + 70000*x2 + 130000*x3 + 160000*x4 <= 15000000)


50000 x1 + 70000 x2 + 130000 x3 + 160000 x4 <= 15000000

In [17]:
model

A JuMP Model
├ solver: Clp
├ objective_sense: MAX_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 4
├ num_constraints: 8
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2
│ ├ AffExpr in MOI.LessThan{Float64}: 2
│ └ VariableRef in MOI.GreaterThan{Float64}: 4
└ Names registered in the model
  └ :x1, :x2, :x3, :x4

In [18]:
# Resolver el problema
optimize!(model)

# Mostrar resultados
println("Estado de la solución: ", termination_status(model))
println("x1 (sencillas): ", value(x1))
println("x2 (dobles): ", value(x2))
println("x3 (triples): ", value(x3))
println("x4 (cuádruples): ", value(x4))
println("Valor objetivo: ", objective_value(model), "\n")

Estado de la solución: OPTIMAL
x1 (sencillas): 18.072289156626503
x2 (dobles): 117.46987951807228
x3 (triples): 45.18072289156626
x4 (cuádruples): 0.0
Valor objetivo: 363253.01204819273

Coin0506I Presolve 4 (0) rows, 4 (0) columns and 16 (0) elements
Clp0006I 0  Obj 0 Dual inf 10492.857 (4)
Clp0006I 5  Obj 363253.01
Clp0000I Optimal - objective value 363253.01
Clp0032I Optimal objective 363253.012 - 5 iterations time 0.002


### 🔢 Enteros

In [19]:
model = Model(Cbc.Optimizer)

@variable(model, x1 >= 0, Int)  # Sencillas
@variable(model, x2 >= 0, Int)  # Dobles
@variable(model, x3 >= 0, Int)  # Triples
@variable(model, x4 >= 0, Int)  # Cuádruples

# Maximizar los impuestos
@objective(model, Max, 1000*x1 + 1900*x2 + 2700*x3 + 3400*x4)

# Restricción del área disponible
@constraint(model, 0.18*x1 + 0.28*x2 + 0.4*x3 + 0.5*x4 <= 63.75)

# Restricción de proporción: Triples y cuádruples >= 25% del total
@constraint(model, x3 + x4 >= 0.25 * (x1 + x2 + x3 + x4))

# Restricción de proporción: Sencillas >= 10% del total
@constraint(model, x1 >= 0.1 * (x1 + x2 + x3 + x4))

# Restricción del financiamiento
@constraint(model, 50000*x1 + 70000*x2 + 130000*x3 + 160000*x4 <= 15000000)

50000 x1 + 70000 x2 + 130000 x3 + 160000 x4 <= 15000000

In [20]:
model

A JuMP Model
├ solver: COIN Branch-and-Cut (Cbc)
├ objective_sense: MAX_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 4
├ num_constraints: 12
│ ├ AffExpr in MOI.GreaterThan{Float64}: 2
│ ├ AffExpr in MOI.LessThan{Float64}: 2
│ ├ VariableRef in MOI.GreaterThan{Float64}: 4
│ └ VariableRef in MOI.Integer: 4
└ Names registered in the model
  └ :x1, :x2, :x3, :x4

In [21]:
# Resolver el problema
optimize!(model)

# Mostrar resultados
println("Estado de la solución: ", termination_status(model))
println("x1 (sencillas): ", round(value(x1)))
println("x2 (dobles): ", round(value(x2)))
println("x3 (triples): ", round(value(x3)))
println("x4 (cuádruples): ", round(value(x4)))
println("Valor objetivo: ", objective_value(model), "\n")

Estado de la solución: OPTIMAL
x1 (sencillas): 18.0
x2 (dobles): 117.0
x3 (triples): 43.0
x4 (cuádruples): 2.0
Valor objetivo: 363200.0

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 363253 - 0.00 seconds
Cgl0004I processed model has 4 rows, 4 columns (4 integer (0 of which binary)) and 16 elements
Cutoff increment increased from 1e-05 to 99.9999
Cbc0012I Integer solution of -363200 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective -363200, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -363253 to -363253
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of 